# First Try

First Try with iCre1355 Models 
Paper: https://onlinelibrary.wiley.com/doi/epdf/10.1111/tpj.13059

In [2]:
from cobra.io import read_sbml_model
from cobra import Reaction, Metabolite
import cobra.util.solver as sutil
from cobra.flux_analysis import flux_variability_analysis as fva
from cobra.exceptions import OptimizationError
import cobra.test
from cobra.flux_analysis import production_envelope
from cobra.flux_analysis.phenotype_phase_plane import production_envelope, add_envelope
from os.path import dirname, join, pardir

from cobra.io import read_sbml_model



In [3]:
from cobra.io import read_sbml_model
hetero = read_sbml_model('iCre1355_hetero.xml')

No objective coefficients in model. Unclear what should be optimized


In [ ]:
mixo = read_sbml_model('iCre1355_mixo.xml')

Model does not contain SBML fbc package information.
SBML package 'layout' not supported by cobrapy, information is not parsed
SBML package 'render' not supported by cobrapy, information is not parsed
Use of FORMULA in the notes element is discouraged, use fbc:chemicalFormula instead: <Species M_10fthf_c "10-Formyltetrahydrofolate">
Use of FORMULA in the notes element is discouraged, use fbc:chemicalFormula instead: <Species M_10fthf_h "10-Formyltetrahydrofolate">
Use of FORMULA in the notes element is discouraged, use fbc:chemicalFormula instead: <Species M_10fthf_m "10-Formyltetrahydrofolate">
Use of FORMULA in the notes element is discouraged, use fbc:chemicalFormula instead: <Species M_10fthf_x "10-Formyltetrahydrofolate">
Use of FORMULA in the notes element is discouraged, use fbc:chemicalFormula instead: <Species M_10fthfglu_L_c "10-Formyltetrahydrofolyl L-glutamate">
Use of FORMULA in the notes element is discouraged, use fbc:chemicalFormula instead: <Species M_10fthfglu_L_m "10

In [ ]:
auto = read_sbml_model('iCre1355_auto.xml')

In [4]:
hetero.compartments

{'c': 'Cytosol',
 'h': 'Chloroplast',
 'm': 'Mitochondria',
 'x': 'Glyoxysome',
 'f': 'Flagellum',
 'e': 'Extra-organism',
 'n': 'Nucleus',
 'g': 'Golgi Apparatus',
 's': 'Eyespot',
 'u': 'Thylakoid Lumen',
 'i': 'Inner Mitochondrial membrane space'}

## Palmitoyl-CoA (n-C16:0CoA) Flux


In [5]:
hetero.metabolites.get_by_id('pmtcoa_x')

Metabolite identifier,pmtcoa_x
Name,Palmitoyl-CoA (n-C16:0CoA)
Memory address,0x07f985b18a588
Formula,C37H62N7O17P3S
Compartment,x
In 2 reaction(s),"ACOA160OR, FA160COAabcp"


### Reactions

In [6]:
hetero.metabolites.pmtcoa_x.summary()

,,PERCENT,FLUX,REACTION_STRING
RXN_STAT,ID,,,
PRODUCING,FA160COAabcp,100.0,0.012767,atp_c + h2o_c + pmtcoa_c --> adp_c + h_c + pi_...
CONSUMING,ACOA160OR,100.0,0.012767,fad_x + pmtcoa_x --> fadh2_x + hdd2coa_x


Palmitoyl-CoA:oxygen 2-oxidoreductase: KEGG
https://www.genome.jp/dbget-bin/www_bget?reaction+R00388+R01175+R01279+R03777+R03857+R03990+R04751+R04754+R07888+R07892+R07896+R07934+R07950

In [17]:
hetero.reactions.get_by_id('ACOA160OR')

Reaction identifier,ACOA160OR
Name,Palmitoyl-CoA:oxygen 2-oxidoreductase
Memory address,0x07f985aa75f98
Stoichiometry,fad_x + pmtcoa_x --> fadh2_x + hdd2coa_x Flavin adenine dinucleotide oxidized + Palmitoyl-CoA (n-C16:0CoA) --> Flavin adenine dinucleotide reduced + trans-Hexadec-2-enoyl-CoA
GPR,Cre16.g687350.t1.2 or Cre16.g689050.t1.1 or Cre16.g695100.t1.1 or Cre05.g232002.t1.1 or Cre05.g23...
Lower bound,0.0
Upper bound,1000.0


### fatty acyl-CoA peroxisomal transport via ABC system (16:0)

FA160COAabcp: transport of pmtcoa from cytoplasma (_c) into the glyxosome (_x) - beta-oxidation

ATP consuming step 

In [16]:
hetero.reactions.get_by_id('FA160COAabcp')

Reaction identifier,FA160COAabcp
Name,fatty acyl-CoA peroxisomal transport via ABC system (16:0)
Memory address,0x07f9859c5be10
Stoichiometry,atp_c + h2o_c + pmtcoa_c --> adp_c + h_c + pi_c + pmtcoa_x ATP + H2O + Palmitoyl-CoA (n-C16:0CoA) --> ADP + H+ + Phosphate + Palmitoyl-CoA (n-C16:0CoA)
GPR,Cre15.g637761.t1.2 or Cre15.g637761.t2.1 or Cre15.g637761.t3.1
Lower bound,0.0
Upper bound,1000.0


In [7]:
hetero.metabolites.atp_c.summary()

In [16]:
mixo.metabolites.pmtcoa_x.summary()

Medium:

In [8]:
hetero.medium

{'EX_h_e': 10.0,
 'EX_h2o_e': 10.0,
 'EX_pi_e': 10.0,
 'EX_nh4_e': 0.5,
 'EX_so4_e': 10.0,
 'EX_fe2_e': 10.0,
 'EX_mg2_e': 10.0,
 'EX_na1_e': 10.0,
 'EX_o2_e': 10.0,
 'EX_ac_e': 2.0}

Changing medium: increasing NH4

Increased activity of FA160COAabcp, increased beta-Oxidation

In [22]:
medium = hetero.medium
with hetero:
    medium['EX_nh4_e'] = 4
    hetero.medium = medium
    hetero.objective = hetero.reactions.FA160COAabcp
    pam_oxidation = hetero.optimize().objective_value
    print("Max. pam oxidation [mmol gDW^-1 h^-1]:", pam_oxidation)
    print("Theoretical max. yield [mmol-succ / mmol-ac]:", pam_oxidation / (-1*hetero.reactions.FA160COAabcp.flux))

Max. pam oxidation [mmol gDW^-1 h^-1]: 0.8440714285714277
Theoretical max. yield [mmol-succ / mmol-ac]: -1.0


In [22]:
hetero.summary() # error

ValueError: Must pass non-zero number of levels/codes

In [11]:
hetero.metabolites.atp_c.summary()

### Changing the medium also affected other pathways for instance:

**ARGSS**: Argininosuccinate Synthase (didn't consume any ATP before -got activated??)
        
**FA40COAabcp**: fatty acyl-CoA peroxisomal transport via ABC system (4:0) -30% percent of ATP (before 4%)

**FTHFL**: Formate--tetrahydrofolate ligase (was active before) https://www.genome.jp/dbget-bin/www_bget?ec:6.3.4.3 (
ec00720  	)Carbon fixation pathways in prokaryotes

                            


    

In [21]:
hetero.reactions.get_by_id('FTHFL')

Reaction identifier,FTHFL
Name,Formate--tetrahydrofolate ligase
Memory address,0x07f985a42bd30
Stoichiometry,"atp_c + for_c + thf_c --> 10fthf_c + adp_c + pi_c ATP + Formate + 5,6,7,8-Tetrahydrofolate --> 10-Formyltetrahydrofolate + ADP + Phosphate"
GPR,Cre13.g566000.t1.2
Lower bound,0.0
Upper bound,1000.0
